In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
!pip install addict

In [ ]:
from addict import Dict
from pathlib import Path

data_dir = Path("/content/drive/MyDrive/Colab Notebooks/Tesi/glacier_mapping-master/datadrive/glaciers")
process_dir = data_dir / "processed"
log_dir = data_dir / "demo/logs"

args = Dict({
    "batch_size": 16,
    "run_name": "demo", 
    "epochs": 200,
    "save_every": 50,
    "loss_type": "dice",
    "device": "cuda:0"
})



In [ ]:
%cd drive/MyDrive/Colab Notebooks/Tesi/glacier_mapping-master

In [ ]:
from glacier_mapping.data.data import fetch_loaders
from glacier_mapping.models.frame import Framework
import glacier_mapping.train as tr
from torch.utils.tensorboard import SummaryWriter
from torchvision.utils import make_grid
from glacier_mapping.models.metrics import diceloss
import yaml
import torch
import json

conf = Dict(yaml.safe_load(open("conf/train.yaml", "r")))
train_folder = "train"
dev_folder = "dev" 
print(process_dir / train_folder)
print(process_dir / dev_folder)
loaders = fetch_loaders(process_dir, args.batch_size, train_folder, dev_folder)
device = torch.device(args.device)

loss_fn = None
outchannels = conf.model_opts.args.outchannels
if args.loss_type == "dice":
    loss_fn = diceloss(
        act=torch.nn.Softmax(dim=1), 
        w=[0.6, 0.9, 0.2], # clean ice, debris, background
        outchannels=outchannels, 
        label_smoothing=0.2
    )
    
frame = Framework(
    model_opts=conf.model_opts,
    optimizer_opts=conf.optim_opts,
    reg_opts=conf.reg_opts,
    device=device,
    loss_fn=loss_fn
)

# Setup logging
writer = SummaryWriter(f"{data_dir}/{args.run_name}/logs/")
writer.add_text("Arguments", json.dumps(vars(args)))
writer.add_text("Configuration Parameters", json.dumps(conf))
out_dir = f"{data_dir}/{args.run_name}/models/"

best_epoch, best_iou = None, 0

for epoch in range(args.epochs):
    loss_d = {}
    loss_d["train"], metrics_train = tr.train_epoch(loaders["train"], frame, conf.metrics_opts)
    tr.log_metrics(writer, metrics_train, loss_d["train"], epoch, "train", mask_names=conf.log_opts.mask_names)
    loss_d["val"], metrics_val = tr.validate(loaders["val"], frame, conf.metrics_opts)
    tr.log_metrics(writer, metrics_val, loss_d["val"], epoch, "val", mask_names=conf.log_opts.mask_names)

    # save model
    writer.add_scalars("Loss", loss_d, epoch)
    if (epoch + 1) % args.save_every == 0:
        frame.save(out_dir, epoch)
        tr.log_images(writer, frame, next(iter(loaders["train"])), epoch)
        tr.log_images(writer, frame, next(iter(loaders["val"])), epoch, "val")

    if best_iou <= metrics_val['IoU'][0]:
        best_iou  = metrics_val['IoU'][0]
        best_epoch = epoch
        frame.save(out_dir, "best")

    print(f"{epoch}/{args.epochs} | train loss: {loss_d['train']} | val loss: {loss_d['val']}")

frame.save(out_dir, "final")
writer.close()

In [ ]:
torch.cuda.is_available()

In [ ]:
conf = Dict(yaml.safe_load(open("conf/train.yaml", "r")))
device = torch.device(args.device)

model_dir = f"{data_dir}/demo/models/model_best.pt"

outchannels = conf.model_opts.args.outchannels

if outchannels > 1:
    loss_weight = [1 for _ in range(outchannels)]
    loss_weight[-1] = 0 # background
    loss_fn = diceloss(act=torch.nn.Softmax(dim=1), w=loss_weight,
                               outchannels=outchannels)
else:
    loss_fn = diceloss()

frame = Framework(
    model_opts=conf.model_opts,
    optimizer_opts=conf.optim_opts,
    reg_opts=conf.reg_opts,
    loss_fn=loss_fn,
    device=device
)
    
unet = frame.model
unet.load_state_dict(torch.load(model_dir))
unet = unet.to(device)

In [ ]:
!pip install torchviz

In [ ]:
x = torch.randn(1,15,512,512).to(device)
y = unet(x)

In [ ]:
#from torchvision import models
from torchsummary import summary

mod = unet
summary(mod, (15, 512, 512), 16)

In [ ]:
from torchviz import make_dot
make_dot(y, params=dict(list(unet.named_parameters())))

In [ ]:
from glacier_mapping.data.data import fetch_loaders
from glacier_mapping.models.frame import Framework
import glacier_mapping.train as tr
from torch.utils.tensorboard import SummaryWriter
from torchvision.utils import make_grid
from glacier_mapping.models.metrics import diceloss
import yaml
import torch
import json
import numpy as np
import os
import matplotlib.pyplot as plt
import time
import glob

In [ ]:

conf = Dict(yaml.safe_load(open("conf/train.yaml", "r")))
device = torch.device(args.device)

model_dir = f"{data_dir}/demo/models/model_best.pt"

outchannels = conf.model_opts.args.outchannels

if outchannels > 1:
    loss_weight = [1 for _ in range(outchannels)]
    loss_weight[-1] = 0 # background
    loss_fn = diceloss(act=torch.nn.Softmax(dim=1), w=loss_weight,
                               outchannels=outchannels)
else:
    loss_fn = diceloss()

frame = Framework(
    model_opts=conf.model_opts,
    optimizer_opts=conf.optim_opts,
    reg_opts=conf.reg_opts,
    loss_fn=loss_fn,
    device=device
)
    
unet = frame.model
unet.load_state_dict(torch.load(model_dir))
unet = unet.to(device)

slices_dir = f"{process_dir}/test/*img*"
pred_dir = f"{process_dir}/preds/"
pred_dir2 = f"{process_dir}/preds2/"
    
if not os.path.exists(pred_dir):
    os.makedirs(pred_dir)

slices = glob.glob(slices_dir)

total_inference_time = 0
for s in slices:
    filename = s.split("/")[-1].replace("npy","png")
    filename2 = s.split("/")[-1]
    inp_np = np.load(s)
    start = time.time()
    nan_mask = np.isnan(inp_np[:,:,:9]).any(axis=2)
    inp_tensor = torch.from_numpy(np.expand_dims(np.transpose(inp_np, (2,0,1)), axis=0))
    inp_tensor = inp_tensor.to(device)
    output = unet(inp_tensor)
    output_np = output.detach().cpu().numpy()
    output_np = np.transpose(output_np[0], (1,2,0))
    output_np = np.argmax(output_np, axis=2)
    output_np[nan_mask] = 3
    total_inference_time += (time.time() - start)
    plt.imsave(f"{pred_dir}{filename}", output_np, vmin=0, vmax=3)
    np.save(f"{pred_dir2}{filename2}", output_np)
    
print(f"Total inference time: {total_inference_time}")

In [ ]:
!pip install rasterio

In [ ]:
import binascii as ba
%pylab inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir datadrive/glaciers/demo/logs

In [ ]:
#load images and use the metrics

import numpy as np
from sklearn.metrics import fbeta_score, accuracy_score, classification_report, confusion_matrix, precision_recall_fscore_support, f1_score, jaccard_score, multilabel_confusion_matrix, ConfusionMatrixDisplay
import rasterio
import os

y_true = list()
y_pred_tmp = list()
y_pred_tmp1 = list()
y_pred_tmp2 = list()
y_pred_tmp3 = list()

pred_png = list()


test_path = "/content/drive/MyDrive/Colab Notebooks/Tesi/glacier_mapping-master/datadrive/glaciers/processed"
true = 'masks'
pred = 'preds2/Unet'

names_images = sorted(os.listdir(os.path.join(test_path,true)))
pred_png = sorted(os.listdir(os.path.join(test_path,pred)))


for image in names_images:
    path = os.path.join(test_path,true,image)
    img_open = np.load(path)
    y_true.append(img_open)

for prediction in pred_png:
    path = os.path.join(test_path,pred,prediction)
    img_open = np.load(path)
    y_pred_tmp.append(img_open)


y_true = np.asarray(y_true)
y_pred = np.asarray(y_pred_tmp)

y_true_tmp = np.zeros_like(y_pred)
y_true_tmp[y_true[:,:,:,0]==1] = 0
y_true_tmp[y_true[:,:,:,1]==1] = 1
y_true_tmp[y_true[:,:,:,2]==1] = 2

y_true = y_true_tmp
print(y_true.shape)
print(y_pred.shape)


y_true = y_true.ravel()
y_pred = y_pred.ravel()

f1_score_teste = f1_score(y_true, y_pred,average='macro')
print("F1-score: ",f1_score_teste)

accuracy_score_teste = accuracy_score(y_true, y_pred)
print("Accuracy: ",accuracy_score_teste)

all_metrics = precision_recall_fscore_support(y_true, y_pred,average="macro")
print("Precision, recall and fscore: ", all_metrics)

iou_score_test = jaccard_score(y_true, y_pred, average=None)
print("IoU: ",iou_score_test)

cf = confusion_matrix(y_true, y_pred)
print("Confusion matrix: ")
print(cf)


fbeta = fbeta_score(y_true, y_pred, average='macro', beta=0.01)
print("Fbeta: ", fbeta)

print(classification_report(y_true, y_pred))

In [ ]:
disp = ConfusionMatrixDisplay(confusion_matrix=cf, display_labels=['Clean Ice', 'Debris', 'Background'])
disp.plot()

In [ ]:
# to visualize percentages in confusion matrix (the sum of each row is equal to 100%)
mat = list()
mat.append(cf[0]/3951515)
mat.append(cf[1]/478519)
mat.append(cf[2]/12871470)
mat = np.array(mat)
disp = ConfusionMatrixDisplay(confusion_matrix=mat, display_labels=['Clean Ice', 'Debris', 'Background'])
disp.plot()

In [ ]:
!pip install git+https://github.com/qubvel/segmentation_models.pytorch

In [ ]:
import segmentation_models_pytorch as smp

In [ ]:
args = Dict({
    "batch_size": 16,
    "run_name": "demo", 
    "epochs": 200,
    "save_every": 50,
    "loss_type": "dice",
    "device": 'cuda:0'
})

In [ ]:
from glacier_mapping.data.data import fetch_loaders
from glacier_mapping.models.frame import Framework
import glacier_mapping.train as tr
from torch.utils.tensorboard import SummaryWriter
from torchvision.utils import make_grid
from glacier_mapping.models.metrics import diceloss
import yaml
import torch
import json

conf = Dict(yaml.safe_load(open("conf/train_fpn.yaml", "r")))
conf.model_opts.name = 'FPN'
train_folder = "train"
dev_folder = "dev"
loaders = fetch_loaders(process_dir, args.batch_size, train_folder, dev_folder)
device = torch.device(args.device)

In [ ]:
#!/usr/bin/env python
"""
Frame to Combine Model with Optimizer

This wraps the model and optimizer objects needed in training, so that each
training step can be concisely called with a single method (optimize).
"""
from pathlib import Path
import os
import torch
import numpy as np
from torch.optim.lr_scheduler import ReduceLROnPlateau
from glacier_mapping.models.metrics import *
from glacier_mapping.models.reg import *
from segmentation_models_pytorch.decoders.fpn.model import *
from segmentation_models_pytorch.decoders.manet.model import *


class Framework2:
    """
    Class to Wrap all the Training Steps

    """

    def __init__(self, loss_fn=None, model_opts=None, optimizer_opts=None,
                 reg_opts=None, device=None):
        """
        Set Class Attrributes
        """
        if device is None:
            self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        else:
            self.device = device
        self.multi_class = True if model_opts.args.classes > 1 else False
        self.num_classes = model_opts.args.classes
        if loss_fn is None:
            if self.multi_class:
                loss_fn = torch.nn.CrossEntropyLoss()
            else:
                loss_fn = torch.nn.BCEWithLogitsLoss()
        self.loss_fn = loss_fn.to(self.device)
        if model_opts.name in ["FPN","MAnet"]:
            model_def = globals()[model_opts.name]
        else:
            raise ValueError("Unknown model name")
        print(model_def)
        self.model = model_def(**model_opts.args).to(self.device)
        optimizer_def = getattr(torch.optim, optimizer_opts.name)
        self.optimizer = optimizer_def(self.model.parameters(), **optimizer_opts.args)
        self.lrscheduler = ReduceLROnPlateau(self.optimizer, "min",
                                             verbose=True, patience=10,
                                             min_lr=1e-6)
        self.reg_opts = reg_opts


    def optimize(self, x, y):
        """
        Take a single gradient step

        Args:
            X: raw training data
            y: labels
        Return:
            optimization
        """
        x = x.permute(0, 3, 1, 2).to(self.device)
        y = y.permute(0, 3, 1, 2).to(self.device)

        self.optimizer.zero_grad()
        y_hat = self.model(x)
        loss = self.calc_loss(y_hat, y)
        loss.backward()
        self.optimizer.step()
        return y_hat.permute(0, 2, 3, 1), loss.item()

    def val_operations(self, val_loss):
        """
        Update the LR Scheduler
        """
        self.lrscheduler.step(val_loss)

    def save(self, out_dir, epoch):
        """
        Save a model checkpoint
        """
        if not os.path.exists(out_dir):
            os.makedirs(out_dir)

        model_path = Path(out_dir, f"model_{epoch}.pt")
        optim_path = Path(out_dir, f"optim_{epoch}.pt")
        torch.save(self.model.state_dict(), model_path)
        torch.save(self.optimizer.state_dict(), optim_path)

    def infer(self, x):
        """ Make a prediction for a given x

        Args:
            x: input x

        Return:
            Prediction

        """
        x = x.permute(0, 3, 1, 2).to(self.device)
        with torch.no_grad():
            return self.model(x).permute(0, 2, 3, 1)

    def segment(self, y_hat):
        """Predict a class given logits

        Args:
            y_hat: logits output

        Return:
            Probability of class in case of binary classification
            or one-hot tensor in case of multi class"""
        if self.multi_class:
            y_hat = torch.argmax(y_hat, axis=3)
            y_hat = torch.nn.functional.one_hot(y_hat, num_classes=self.num_classes)
        else:
            y_hat = torch.sigmoid(y_hat)
        return y_hat

    def act(self, logits):
        """Applies activation function based on the model
        Args:
            y_hat: logits output
        Returns:
            logits after applying activation function"""

        if self.multi_class:
            y_hat = torch.nn.Softmax(3)(logits)
        else:
            y_hat = torch.sigmoid(logits)
        return y_hat

    def calc_loss(self, y_hat, y):
        """ Compute loss given a prediction

        Args:
            y_hat: Prediction
            y: Label

        Return:
            Loss values

        """
        y_hat = y_hat.to(self.device)
        y = y.to(self.device)

        if self.multi_class:
            y = torch.argmax(y, dim=1)
            y = torch.tensor(y, dtype=torch.long, device=self.device)

        loss = self.loss_fn(y_hat, y)

        for reg_type in self.reg_opts.keys():
            reg_fun = globals()[reg_type]
            penalty = reg_fun(
                self.model.parameters(),
                self.reg_opts[reg_type],
                self.device
            )
            loss += penalty

        return loss


    def metrics(self, y_hat, y, metrics_opts):
        """ Loop over metrics in train.yaml

        Args:
            y_hat: Predictions
            y: Labels
            metrics_opts: Metrics specified in the train.yaml

        Return:
            results

        """
        y_hat = y_hat.to(self.device)
        y = y.to(self.device)

        results = {}
        for k, metric in metrics_opts.items():
            if "threshold" in metric.keys():
                y_hat = y_hat > metric["threshold"]

            metric_fun = globals()[k]
            results[k] = metric_fun(y_hat, y)
        return results


In [ ]:
loss_fn = None
outchannels = conf.model_opts.args.classes
if args.loss_type == "dice":
    loss_fn = diceloss(
        act=torch.nn.Softmax(dim=1), 
        w=[0.6, 0.9, 0.2], # clean ice, debris, background
        outchannels=outchannels, 
        label_smoothing=0.2
    )
    
frame = Framework2(
    model_opts=conf.model_opts,
    optimizer_opts=conf.optim_opts,
    reg_opts=conf.reg_opts,
    device=device,
    loss_fn=loss_fn
)

# encoder name
enc_name = conf.model_opts.args.encoder_name

# Setup logging
writer = SummaryWriter(f"{data_dir}/{args.run_name}/FPN/{enc_name}/logs/")
writer.add_text("Arguments", json.dumps(vars(args)))
writer.add_text("Configuration Parameters", json.dumps(conf))
out_dir = f"{data_dir}/{args.run_name}/models/FPN/{enc_name}/"

best_epoch, best_iou = None, 0
for epoch in range(args.epochs):
  loss_d = {}
  loss_d["train"], metrics_train = tr.train_epoch(loaders["train"], frame, conf.metrics_opts)
  tr.log_metrics(writer, metrics_train, loss_d["train"], epoch, "train", mask_names=conf.log_opts.mask_names)
  loss_d["val"], metrics_val = tr.validate(loaders["val"], frame, conf.metrics_opts)
  tr.log_metrics(writer, metrics_val, loss_d["val"], epoch, "val", mask_names=conf.log_opts.mask_names)

  # save model
  writer.add_scalars("Loss", loss_d, epoch)
  if (epoch + 1) % args.save_every == 0:
      frame.save(out_dir, epoch)
      tr.log_images(writer, frame, next(iter(loaders["train"])), epoch)
      tr.log_images(writer, frame, next(iter(loaders["val"])), epoch, "val")

  if best_iou <= metrics_val['IoU'][0]:
      best_iou  = metrics_val['IoU'][0]
      best_epoch = epoch
      frame.save(out_dir, "best")

  print(f"{epoch}/{args.epochs} | train loss: {loss_d['train']} | val loss: {loss_d['val']}")

frame.save(out_dir, "final")
writer.close()

In [ ]:
from glacier_mapping.data.data import fetch_loaders
import glacier_mapping.train as tr
from torch.utils.tensorboard import SummaryWriter
from torchvision.utils import make_grid
from glacier_mapping.models.metrics import diceloss
import yaml
import torch
import json
import numpy as np
import os
import matplotlib.pyplot as plt
import time
import glob

In [ ]:
conf = Dict(yaml.safe_load(open("conf/train_fpn.yaml", "r")))
conf.model_opts.name = 'FPN'
device = torch.device(args.device)

model_dir = f"{data_dir}/demo/models/FPN/vgg19/model_best.pt"

outchannels = conf.model_opts.args.classes

if outchannels > 1:
    loss_weight = [1 for _ in range(outchannels)]
    loss_weight[-1] = 0 # background
    loss_fn = diceloss(act=torch.nn.Softmax(dim=1), w=loss_weight,
                               outchannels=outchannels)
else:
    loss_fn = diceloss()

frame = Framework2(
    model_opts=conf.model_opts,
    optimizer_opts=conf.optim_opts,
    reg_opts=conf.reg_opts,
    loss_fn=loss_fn,
    device=device
)
    
fpn = frame.model
fpn.load_state_dict(torch.load(model_dir))#, map_location=torch.device('cpu')
fpn = fpn.to(device)

In [ ]:
from torch.utils.tensorboard import SummaryWriter

writer = SummaryWriter(f"{data_dir}/{args.run_name}/logs/model_plot/fpn/")
x = torch.randn(16,15,512,512).to(device)
writer.add_graph(fpn, x)
writer.close()

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir datadrive/glaciers/demo/logs/model_plot/fpn

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir datadrive/glaciers/demo/FPN/vgg19/logs

In [ ]:
slices_dir = f"{process_dir}/test/*img*"
pred_dir = f"{process_dir}/preds/FPN/"
pred_dir2 = f"{process_dir}/preds2/FPN/"
    
if not os.path.exists(pred_dir):
    os.makedirs(pred_dir)

slices = glob.glob(slices_dir)

total_inference_time = 0
for s in slices:
    filename = s.split("/")[-1].replace("npy","png")
    filename2 = s.split("/")[-1]
    inp_np = np.load(s)
    start = time.time()
    nan_mask = np.isnan(inp_np[:,:,:9]).any(axis=2)
    inp_tensor = torch.from_numpy(np.expand_dims(np.transpose(inp_np, (2,0,1)), axis=0))
    inp_tensor = inp_tensor.to(device)
    output = fpn(inp_tensor)
    output_np = output.detach().cpu().numpy()
    output_np = np.transpose(output_np[0], (1,2,0))
    output_np = np.argmax(output_np, axis=2)
    output_np[nan_mask] = 3
    total_inference_time += (time.time() - start)
    plt.imsave(f"{pred_dir}{filename}", output_np, vmin=0, vmax=3)
    np.save(f"{pred_dir2}{filename2}", output_np)
print(f"Total inference time: {total_inference_time}")

In [ ]:
import numpy as np
from sklearn.metrics import fbeta_score, accuracy_score, classification_report, confusion_matrix, precision_recall_fscore_support, f1_score, multilabel_confusion_matrix, ConfusionMatrixDisplay, jaccard_score
import rasterio
import os

In [ ]:
#load images and use the metrics
y_true = list()
y_pred_tmp = list()
y_pred_tmp1 = list()
y_pred_tmp2 = list()
y_pred_tmp3 = list()

pred_png = list()


test_path = "/content/drive/MyDrive/Colab Notebooks/Tesi/glacier_mapping-master/datadrive/glaciers/processed"
true = 'masks'
pred = 'preds2/FPN'

names_images = sorted(os.listdir(os.path.join(test_path,true)))
pred_png = sorted(os.listdir(os.path.join(test_path,pred)))

for image in names_images:
    path = os.path.join(test_path,true,image)
    img_open = np.load(path)
    y_true.append(img_open)


for prediction in pred_png:
    path = os.path.join(test_path,pred,prediction)
    img_open = np.load(path)
    y_pred_tmp.append(img_open)

y_true = np.asarray(y_true)
y_pred = np.asarray(y_pred_tmp)

y_true_tmp = np.zeros_like(y_pred)
y_true_tmp[y_true[:,:,:,0]==1] = 0
y_true_tmp[y_true[:,:,:,1]==1] = 1
y_true_tmp[y_true[:,:,:,2]==1] = 2

y_true = y_true_tmp


y_true = y_true.ravel()
y_pred = y_pred.ravel()

f1_score_teste = f1_score(y_true, y_pred,average='macro')
print("F1-score: ",f1_score_teste)

accuracy_score_teste = accuracy_score(y_true, y_pred)
print("Accuracy: ",accuracy_score_teste)

all_metrics = precision_recall_fscore_support(y_true, y_pred,average="macro")
print("Precision, recall and fscore: ", all_metrics)

iou_score_test = jaccard_score(y_true, y_pred, average=None)
print("IoU: ",iou_score_test)

cf = confusion_matrix(y_true, y_pred)
print("Confusion matrix: ")
print(cf)


fbeta = fbeta_score(y_true, y_pred, average='macro', beta=0.01)
print("Fbeta: ", fbeta)

print(classification_report(y_true, y_pred))

In [ ]:
disp = ConfusionMatrixDisplay(confusion_matrix=cf, display_labels=['Clean Ice', 'Debris', 'Background'])
disp.plot()

In [ ]:
# to visualize percentages in confusion matrix (the sum of each row is equal to 100%)
mat = list()
mat.append(cf[0]/3951515)
mat.append(cf[1]/478519)
mat.append(cf[2]/12871470)
mat = np.array(mat)
disp = ConfusionMatrixDisplay(confusion_matrix=mat, display_labels=['Clean Ice', 'Debris', 'Background'])
disp.plot()

In [ ]:
from glacier_mapping.data.data import fetch_loaders
import glacier_mapping.train as tr
from torch.utils.tensorboard import SummaryWriter
from torchvision.utils import make_grid
from glacier_mapping.models.metrics import diceloss
import yaml
import torch
import json

conf = Dict(yaml.safe_load(open("conf/train_manet.yaml", "r")))
conf.model_opts.name = 'MAnet'
train_folder = "train"
dev_folder = "dev"
loaders = fetch_loaders(process_dir, args.batch_size, train_folder, dev_folder)
device = torch.device(args.device)

In [ ]:
loss_fn = None
outchannels = conf.model_opts.args.classes
if args.loss_type == "dice":
    loss_fn = diceloss(
        act=torch.nn.Softmax(dim=1), 
        w=[0.6, 0.9, 0.2], # clean ice, debris, background
        outchannels=outchannels, 
        label_smoothing=0.2
    )
    
frame = Framework2(
    model_opts=conf.model_opts,
    optimizer_opts=conf.optim_opts,
    reg_opts=conf.reg_opts,
    device=device,
    loss_fn=loss_fn
)

# encoder name
enc_name = conf.model_opts.args.encoder_name

# Setup logging
writer = SummaryWriter(f"{data_dir}/{args.run_name}/MANet/{enc_name}/logs/")
writer.add_text("Arguments", json.dumps(vars(args)))
writer.add_text("Configuration Parameters", json.dumps(conf))
out_dir = f"{data_dir}/{args.run_name}/models/MANet/{enc_name}/"

best_epoch, best_iou = None, 0
for epoch in range(args.epochs):
  loss_d = {}
  loss_d["train"], metrics_train = tr.train_epoch(loaders["train"], frame, conf.metrics_opts)
  tr.log_metrics(writer, metrics_train, loss_d["train"], epoch, "train", mask_names=conf.log_opts.mask_names)
  loss_d["val"], metrics_val = tr.validate(loaders["val"], frame, conf.metrics_opts)
  tr.log_metrics(writer, metrics_val, loss_d["val"], epoch, "val", mask_names=conf.log_opts.mask_names)

  # save model
  writer.add_scalars("Loss", loss_d, epoch)
  if (epoch + 1) % args.save_every == 0:
      frame.save(out_dir, epoch)
      tr.log_images(writer, frame, next(iter(loaders["train"])), epoch)
      tr.log_images(writer, frame, next(iter(loaders["val"])), epoch, "val")

  if best_iou <= metrics_val['IoU'][0]:
      best_iou  = metrics_val['IoU'][0]
      best_epoch = epoch
      frame.save(out_dir, "best")

  print(f"{epoch}/{args.epochs} | train loss: {loss_d['train']} | val loss: {loss_d['val']}")

frame.save(out_dir, "final")
writer.close()

In [ ]:
from glacier_mapping.data.data import fetch_loaders
from glacier_mapping.models.frame import Framework
import glacier_mapping.train as tr
from torch.utils.tensorboard import SummaryWriter
from torchvision.utils import make_grid
from glacier_mapping.models.metrics import diceloss
import yaml
import torch
import json
import numpy as np
import os
import matplotlib.pyplot as plt
import time
import glob

In [ ]:
conf = Dict(yaml.safe_load(open("conf/train_manet.yaml", "r")))
conf.model_opts.name = 'MAnet'
device = torch.device(args.device)

model_dir = f"{data_dir}/demo/models/MANet/vgg16/model_best.pt"

outchannels = conf.model_opts.args.classes

if outchannels > 1:
    loss_weight = [1 for _ in range(outchannels)]
    loss_weight[-1] = 0 # background
    loss_fn = diceloss(act=torch.nn.Softmax(dim=1), w=loss_weight,
                               outchannels=outchannels)
else:
    loss_fn = diceloss()

frame = Framework2(
    model_opts=conf.model_opts,
    optimizer_opts=conf.optim_opts,
    reg_opts=conf.reg_opts,
    loss_fn=loss_fn,
    device=device
)
    
manet = frame.model
manet.load_state_dict(torch.load(model_dir))#, map_location=torch.device('cpu')
manet = manet.to(device)

In [ ]:
from torch.utils.tensorboard import SummaryWriter

writer = SummaryWriter(f"{data_dir}/{args.run_name}/logs/model_plot/manet/")
x = torch.randn(16,15,512,512).to(device)
writer.add_graph(manet, x)
writer.close()

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir datadrive/glaciers/demo/logs/model_plot/manet

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir datadrive/glaciers/demo/MANet/vgg16/logs

In [ ]:
slices_dir = f"{process_dir}/test/*img*"
pred_dir = f"{process_dir}/preds/MANet/vgg16/"
pred_dir2 = f"{process_dir}/preds2/MANet/vgg16/"
    
if not os.path.exists(pred_dir):
    os.makedirs(pred_dir)

slices = glob.glob(slices_dir)

total_inference_time = 0
for s in slices:
    filename = s.split("/")[-1].replace("npy","png")
    filename2 = s.split("/")[-1]
    inp_np = np.load(s)
    start = time.time()
    nan_mask = np.isnan(inp_np[:,:,:9]).any(axis=2)
    inp_tensor = torch.from_numpy(np.expand_dims(np.transpose(inp_np, (2,0,1)), axis=0))
    inp_tensor = inp_tensor.to(device)
    output = manet(inp_tensor)
    output_np = output.detach().cpu().numpy()
    output_np = np.transpose(output_np[0], (1,2,0))
    output_np = np.argmax(output_np, axis=2)
    output_np[nan_mask] = 3
    total_inference_time += (time.time() - start)
    plt.imsave(f"{pred_dir}{filename}", output_np, vmin=0, vmax=3)
    np.save(f"{pred_dir2}{filename2}", output_np)
print(f"Total inference time: {total_inference_time}")

In [ ]:
import numpy as np
from sklearn.metrics import fbeta_score, accuracy_score, classification_report, confusion_matrix, precision_recall_fscore_support, f1_score, jaccard_score, multilabel_confusion_matrix, ConfusionMatrixDisplay
import rasterio
import os

In [ ]:
#load images and use the metrics
y_true = list()
y_pred_tmp = list()
y_pred_tmp1 = list()
y_pred_tmp2 = list()
y_pred_tmp3 = list()

pred_png = list()


test_path = "/content/drive/MyDrive/Colab Notebooks/Tesi/glacier_mapping-master/datadrive/glaciers/processed"
true = 'masks'
pred = 'preds2/MANet/vgg16'

names_images = sorted(os.listdir(os.path.join(test_path,true)))
pred_png = sorted(os.listdir(os.path.join(test_path,pred)))

for image in names_images:
    path = os.path.join(test_path,true,image)
    img_open = np.load(path)
    y_true.append(img_open)

for prediction in pred_png:
    path = os.path.join(test_path,pred,prediction)
    img_open = np.load(path)
    y_pred_tmp.append(img_open)

y_true = np.asarray(y_true)
y_pred = np.asarray(y_pred_tmp)

y_true_tmp = np.zeros_like(y_pred)
y_true_tmp[y_true[:,:,:,0]==1] = 0
y_true_tmp[y_true[:,:,:,1]==1] = 1
y_true_tmp[y_true[:,:,:,2]==1] = 2

y_true = y_true_tmp


y_true = y_true.ravel()
y_pred = y_pred.ravel()

f1_score_teste = f1_score(y_true, y_pred,average='macro')
print("F1-score: ",f1_score_teste)

accuracy_score_teste = accuracy_score(y_true, y_pred)
print("Accuracy: ",accuracy_score_teste)

all_metrics = precision_recall_fscore_support(y_true, y_pred,average="macro")
print("Precision, recall and fscore: ", all_metrics)

iou_score_test = jaccard_score(y_true, y_pred, average=None)
print("IoU: ",iou_score_test)

cf = confusion_matrix(y_true, y_pred)
print("Confusion matrix: ")
print(cf)


fbeta = fbeta_score(y_true, y_pred, average='macro', beta=0.01)
print("Fbeta: ", fbeta)

print(classification_report(y_true, y_pred))

In [ ]:
disp = ConfusionMatrixDisplay(confusion_matrix=cf, display_labels=['Clean Ice', 'Debris', 'Background'])
disp.plot()
mat = list()

In [ ]:
# to visualize percentages in confusion matrix (the sum of each row is equal to 100%)
mat.append(cf[0]/3951515)
mat.append(cf[1]/478519)
mat.append(cf[2]/12871470)
mat = np.array(mat)
disp = ConfusionMatrixDisplay(confusion_matrix=mat, display_labels=['Clean Ice', 'Debris', 'Background'])
disp.plot()

In [ ]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
import pickle
import os
import time
import glob

In [ ]:
train_features, train_labels = next(iter(loaders['train']))
print(f"Feature batch shape: {train_features.size()}")
print(f"Labels batch shape: {train_labels.size()}")

In [ ]:
b_size, h, w, _ = train_features.shape
train_features = train_features.reshape(b_size*h*w, -1)
train_labels = train_labels.reshape(b_size*h*w, -1)

In [ ]:
clf = RandomForestClassifier(n_estimators=50)
for load in iter(loaders['train']):
  train_features, train_labels = load
  del load
  print(train_features.shape)
  b_size, h, w, _ = train_features.shape
  train_features = train_features.reshape(b_size*h*w, -1)
  train_labels = train_labels.reshape(b_size*h*w, -1)
  clf.fit(train_features,train_labels)
  del train_features
  del train_labels
#clf.save(out_dir, "randomforest")

In [ ]:
clf = RandomForestClassifier(n_estimators=50)
clf.fit(train_features,train_labels)
filename = '/content/drive/MyDrive/Colab Notebooks/Tesi/glacier_mapping-master/random_forest.sav'
pickle.dump(clf, open(filename, 'wb'))

In [ ]:
filename = '/content/drive/MyDrive/Colab Notebooks/Tesi/glacier_mapping-master/random_forest.sav'
loaded_model = pickle.load(open(filename, 'rb'))

In [ ]:
from matplotlib.image import imread
import matplotlib.pyplot as plt

In [ ]:
labels = ['Blue', 'Green', 'Red', 'NIR', 'SWIR1', 'Low-Gain TIR', 'High-Gain TIR', 'SWIR2', 'Panchromatic', 'Quality Bitmask', 'NDVI', 'NDSI', 'NDWI', 'Elevation', 'Slope']

In [ ]:
sorted_idx = loaded_model.feature_importances_.argsort()
plt.barh(labels, loaded_model.feature_importances_)
plt.xlabel("Random Forest Feature Importance")

In [ ]:
slices_dir = f"{process_dir}/test/*img*"
pred_dir = f"{process_dir}/preds/RandomForest/"
pred_dir2 = f"{process_dir}/preds2/RandomForest/"
    
if not os.path.exists(pred_dir):
    os.makedirs(pred_dir)

if not os.path.exists(pred_dir2):
    os.makedirs(pred_dir2)

slices = glob.glob(slices_dir)

total_inference_time = 0
for s in slices:
    filename = s.split("/")[-1].replace("npy","png")
    filename2 = s.split("/")[-1]
    inp_np = np.load(s)
    input = inp_np.reshape(-1,15)
    start = time.time()
    y_pred = loaded_model.predict(input)
    classification = y_pred.reshape(512,512,3)
    total_inference_time += (time.time() - start)
    plt.imsave(f"{pred_dir}{filename}", classification, vmin=0, vmax=3)
    np.save(f"{pred_dir2}{filename2}", classification)
print(f"Total inference time: {total_inference_time}")

In [ ]:
print(classification.shape)
y_true_tmp = np.zeros_like(classification)
y_true_tmp[classification[:,:,0]==1] = 0
y_true_tmp[classification[:,:,1]==1] = 1
y_true_tmp[classification[:,:,2]==1] = 2
print(y_true_tmp.shape)
plt.imshow(y_true_tmp[...,2])

In [ ]:
!pip install rasterio

In [ ]:
import numpy as np
from sklearn.metrics import fbeta_score, accuracy_score, classification_report, confusion_matrix, precision_recall_fscore_support, f1_score, multilabel_confusion_matrix, ConfusionMatrixDisplay, jaccard_score
import rasterio
import os

In [ ]:
#load images and use the metrics
y_true = list()
y_pred_tmp = list()
y_pred_tmp1 = list()
y_pred_tmp2 = list()
y_pred_tmp3 = list()

pred_png = list()


test_path = "/content/drive/MyDrive/Colab Notebooks/Tesi/glacier_mapping-master/datadrive/glaciers/processed"
true = 'masks'
pred = 'preds2/RandomForest'

names_images = sorted(os.listdir(os.path.join(test_path,true)))
pred_png = sorted(os.listdir(os.path.join(test_path,pred)))

for image in names_images:
    path = os.path.join(test_path,true,image)
    img_open = np.load(path)
    y_true.append(img_open)

for prediction in pred_png:
    path = os.path.join(test_path,pred,prediction)
    img_open = np.load(path)
    y_pred_tmp.append(img_open)

y_true = np.asarray(y_true)
y_pred = np.asarray(y_pred_tmp)

y_true_tmp = np.zeros_like(y_pred)
y_true_tmp[y_true[:,:,:,0]==1] = 0
y_true_tmp[y_true[:,:,:,1]==1] = 1
y_true_tmp[y_true[:,:,:,2]==1] = 2

y_true = y_true_tmp[...,0]


y_pred_tmp = np.zeros_like(y_pred)
y_pred_tmp[y_pred[:,:,:,0]==1] = 0
y_pred_tmp[y_pred[:,:,:,1]==1] = 1
y_pred_tmp[y_pred[:,:,:,2]==1] = 2

y_pred = y_pred_tmp[...,0]

y_true = y_true.ravel()
y_pred = y_pred.ravel()

f1_score_teste = f1_score(y_true, y_pred,average='macro')
print("F1-score: ",f1_score_teste)

accuracy_score_teste = accuracy_score(y_true, y_pred)
print("Accuracy: ",accuracy_score_teste)

iou_score_test = jaccard_score(y_true, y_pred, average=None)
print("IoU: ",iou_score_test)

all_metrics = precision_recall_fscore_support(y_true, y_pred,average="macro")
print("Precision, recall and fscore: ", all_metrics)

cf = confusion_matrix(y_true, y_pred)
print("Confusion matrix: ")
print(cf)


fbeta = fbeta_score(y_true, y_pred, average='macro', beta=0.01)
print("Fbeta: ", fbeta)

print(classification_report(y_true, y_pred))

In [ ]:
disp = ConfusionMatrixDisplay(confusion_matrix=cf, display_labels=['Clean Ice', 'Debris', 'Background'])
disp.plot()

In [ ]:
# to visualize percentages in confusion matrix (the sum of each row is equal to 100%)
mat = list()
mat.append(cf[0]/3951515)
mat.append(cf[1]/478519)
mat.append(cf[2]/12871470)
mat = np.array(mat)
disp = ConfusionMatrixDisplay(confusion_matrix=mat, display_labels=['Clean Ice', 'Debris', 'Background'])
disp.plot()

In [ ]:
!pip install earthpy

In [ ]:
from matplotlib.image import imread
import matplotlib.pyplot as plt

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import earthpy.plot as ep

img_array = np.load('/content/drive/MyDrive/Colab Notebooks/Tesi/glacier_mapping-master/datadrive/glaciers/processed/test/slice_8_img_132.npy')
mask_array = np.load('/content/drive/MyDrive/Colab Notebooks/Tesi/glacier_mapping-master/datadrive/glaciers/processed/test/slice_8_mask_132.npy')

f, axarr = plt.subplots(3,5)
axarr[0,0].imshow(img_array[...,0])
axarr[0,1].imshow(img_array[...,1])
axarr[0,2].imshow(img_array[...,2])
axarr[0,3].imshow(img_array[...,3])
axarr[0,4].imshow(img_array[...,4])

axarr[1,0].imshow(img_array[...,5])
axarr[1,1].imshow(img_array[...,6])
axarr[1,2].imshow(img_array[...,7])
axarr[1,3].imshow(img_array[...,8])
axarr[1,4].imshow(img_array[...,9])

axarr[2,0].imshow(img_array[...,10])
axarr[2,1].imshow(img_array[...,11])
axarr[2,2].imshow(img_array[...,12])
axarr[2,3].imshow(img_array[...,13])
axarr[2,4].imshow(img_array[...,14])


f, bxarr = plt.subplots(1,3)
bxarr[0].imshow(mask_array[...,0])
bxarr[1].imshow(mask_array[...,1])
bxarr[2].imshow(mask_array[...,2])

mask = np.zeros_like(mask_array)
mask[mask_array[:,:,0]==1] = 0
mask[mask_array[:,:,1]==1] = 1
mask[mask_array[:,:,2]==1] = 2
print(mask.shape)
f = plt.figure()
plt.imshow(mask[...,0])
plt.show()


image = np.dstack((img_array[...,2],img_array[...,1],img_array[...,0]))


image2 = image.transpose(2,0,1)
ep.plot_rgb(image2,
            title="RGB Composite Image",
            stretch=True,
            str_clip=1)
plt.show()

In [ ]:
img_arr = np.load('/content/drive/MyDrive/Colab Notebooks/Tesi/glacier_mapping-master/datadrive/glaciers/processed/test/slice_2_img_109.npy')

print('The shape of the image is ', img_arr.shape)

image = np.dstack((img_arr[...,10],img_arr[...,11],img_arr[...,12]))
print('The shape of the image (only indices) is ', image.shape)

In [ ]:
plt.figure(figsize=(12,12))
# changing the index in the square brackets you can visualize the 15 bands
plt.imshow(img_arr[...,0])

In [ ]:
x_img = img_arr.reshape(-1,15)
print('The shape after reshaping of the image is ', x_img.shape)

x_image = image.reshape(-1,3)
print('The shape after reshaping of the image (only indices) is ', x_image.shape)

In [ ]:
from sklearn.cluster import KMeans, DBSCAN

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_features = scaler.fit_transform(x_img)
scaled_features

In [ ]:
kmeans_kwargs = {
     "init": "random",
     "n_init": 10,
     "max_iter": 300,
     "random_state": 42,
}
# A list holds the SSE values for each k
sse = []
for k in range(1, 11):
    kmeans = KMeans(n_clusters=k, **kmeans_kwargs)
    kmeans.fit(scaled_features)
    sse.append(kmeans.inertia_)

In [ ]:
plt.style.use("fivethirtyeight")
plt.plot(range(1, 11), sse)
plt.xticks(range(1, 11))
plt.xlabel("Number of Clusters")
plt.ylabel("SSE")
plt.show()

In [ ]:
km = KMeans(5)
km.fit(x_img)

In [ ]:
seg = km.predict(x_img).reshape(img_arr.shape[:-1])
seg

In [ ]:
plt.imshow(seg)